**Create a custom dataframe**


# Inflation

Inflation model for analyzing, tracking, and predicting components of inflation.

There are many factors that contribute to the act or process of _Consumer Inflation;_ however, there are only a few methods created for measuring the average change of indexes over time that are strong indicators a possible period of inflation.

One of those indexes is the _[Consumer Price Index](https://www.bls.gov/cpi/)_ (CPI) and the _[Consumer Expenditure Survey](https://data.bls.gov/cex/)_ (CES). CPI is a measure of the price for goods and services over a period of time, while CES is a measure of spending per household. 

**KEY** <br> Within this analysis all datasets will include the following endings:
-    **U** = All urban consumers<br> 
-    **W** = All urban wages<br> 
-    **NF** = Nonfarm cpi<br> 
-    **Gov** = Government

In [16]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split

In [66]:
# Read in our datasets
print(f'Shape of our datasets:','\n','CES for Government Employees:',ces_gov.shape,'\n',
      'Consumer Price Index for All Urban:',cpi_u.shape,'\n',
      'Consumer Price Index for All Wages:', cpi_w.shape,'\n',
      'Consumer Expenditure Survey Nonfarm:', ces_total_nf.shape),'\n',
ces_gov = pd.read_csv('/Users/jasonrobinson/Documents/Projects/bls_project/data_2/SeriesReport-ces-government.csv')
cpi_u = pd.read_csv('/Users/jasonrobinson/Documents/Projects/bls_project/data_2/SeriesReport-cpi-all-u-notadj.48.csv')
cpi_w = pd.read_csv('/Users/jasonrobinson/Documents/Projects/bls_project/data_2/SeriesReport-cpi-urbwage-notadj.48.csv')
ces_total_nf = pd.read_csv('/Users/jasonrobinson/Documents/Projects/bls_project/data_2/SeriesReport-ces-total-nonfarm.csv')

Shape of our datasets: 
 CES for Government Employees: (11, 13) 
 Consumer Price Index for All Urban: (75, 13) 
 Consumer Price Index for All Wages: (75, 0) 
 Consumer Expenditure Survey Nonfarm: (11, 13)


In [67]:
cpi_w.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
0,1948,23.8,23.6,23.6,23.9,24.1,24.2,24.5,24.6,24.6,24.5,24.4,24.2,NaN,NaN
1,1949,24.2,23.9,24.0,24.0,24.0,24.0,23.8,23.9,24.0,23.9,23.9,23.8,NaN,NaN
2,1950,23.7,23.6,23.7,23.7,23.8,24.0,24.2,24.4,24.6,24.7,24.8,25.1,NaN,NaN
3,1951,25.5,25.9,26.0,26.0,26.1,26.1,26.1,26.1,26.3,26.4,26.5,26.6,NaN,NaN
4,1952,26.6,26.5,26.5,26.6,26.6,26.7,26.9,26.9,26.9,26.9,26.9,26.9,NaN,NaN


In [72]:
# Since cpi-w is not the same size as our cpi-u dataset we can easily drop
# the two extra columns as they are not necessary to the training
#cpi_w = cpi_w.drop(cpi_w[['HALF1', 'HALF2']], axis=1)
print(cpi_w.shape)
cpi_w.head()

(75, 13)


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948,23.8,23.6,23.6,23.9,24.1,24.2,24.5,24.6,24.6,24.5,24.4,24.2
1,1949,24.2,23.9,24.0,24.0,24.0,24.0,23.8,23.9,24.0,23.9,23.9,23.8
2,1950,23.7,23.6,23.7,23.7,23.8,24.0,24.2,24.4,24.6,24.7,24.8,25.1
3,1951,25.5,25.9,26.0,26.0,26.1,26.1,26.1,26.1,26.3,26.4,26.5,26.6
4,1952,26.6,26.5,26.5,26.6,26.6,26.7,26.9,26.9,26.9,26.9,26.9,26.9


**Get the summary statistics** 

In [77]:
# Work with all urban and urban/wages
print(cpi_w.describe())
cpi_u.describe(include='all')

              Year         Jan         Feb         Mar         Apr  \
count    75.000000   75.000000   74.000000   74.000000   74.000000   
mean   1985.000000  113.994147  112.214919  112.725324  113.180257   
std      21.794495   79.448563   77.989041   78.410040   78.700877   
min    1948.000000   23.700000   23.600000   23.600000   23.700000   
25%    1966.500000   32.550000   32.425000   32.525000   32.700000   
50%    1985.000000  104.900000  103.600000  103.850000  104.200000   
75%    2003.500000  179.300000  177.825000  178.900000  178.800000   
max    2022.000000  276.296000  256.843000  258.935000  261.237000   

              May         Jun         Jul         Aug         Sep         Oct  \
count   74.000000   74.000000   74.000000   74.000000   74.000000   74.000000   
mean   113.572203  113.956986  114.159473  114.417946  114.762595  114.894743   
std     78.957409   79.158330   79.184861   79.262302   79.407066   79.365418   
min     23.800000   24.000000   23.800000   2

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
count,75.000000,75.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000
mean,1985.000000,115.730027,113.941081,114.453311,114.868554,115.230743,115.610919,115.837108,116.088851,116.419419,116.599351,116.572797,116.522838
std,21.794495,81.627283,80.171231,80.567981,80.818227,81.044034,81.233918,81.268215,81.352420,81.489663,81.484002,81.301846,81.130291
min,1948.000000,23.500000,23.500000,23.400000,23.600000,23.700000,23.800000,23.700000,23.800000,23.900000,23.700000,23.800000,23.600000
25%,1966.500000,32.350000,32.225000,32.325000,32.500000,32.525000,32.625000,32.725000,32.900000,32.925000,33.100000,33.125000,33.150000
50%,1985.000000,105.500000,104.200000,104.500000,105.000000,105.350000,105.650000,105.950000,106.250000,106.650000,107.000000,107.150000,107.300000
75%,2003.500000,183.450000,181.775000,182.850000,182.800000,182.575000,182.750000,182.950000,183.625000,184.150000,184.075000,183.700000,183.450000
max,2022.000000,281.148000,263.014000,264.877000,267.054000,269.195000,271.696000,273.003000,273.567000,274.310000,276.589000,277.948000,278.802000


In [110]:
fig = px.histogram(cpi_u["Year"], x=cpi_u['Jan'], nbins=50)
fig.show()

In [111]:
fig = px.pairplot(cpi_u, diag_kind="kde")
fig.show()

AttributeError: module 'plotly.express' has no attribute 'pairplot'

As we would expect, all values share closely related means and standared deviations, but we want to drill down on the variance of the distributions.

In [113]:
fig = px.scatter(cpi_u, x=cpi_u["Jan"], y="May", color="Jun", size="Aug")
fig.show()

/opt/anaconda3/lib/python3.8/site-packages/_plotly_utils/basevalidators.py:776: RuntimeWarning:

invalid value encountered in greater_equal

/opt/anaconda3/lib/python3.8/site-packages/_plotly_utils/basevalidators.py:776: RuntimeWarning:

invalid value encountered in less_equal



ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [nan]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above


## Model Development


In [2]:
import tensorflow as tf

In [52]:
# Range of time series is 73 years period 1948-2021
cpi = pd.read_csv('/Users/jasonrobinson/Documents/Projects/bls_project/data_2/SeriesReport-cpi-urbwage-notadj.48.csv',
                 index_col='Year')

# Drop unnecessary columns
cpi = cpi.drop(['HALF1', 'HALF2'], axis=1)
cpi.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Year,,,,,,,,,,,,
1948,23.8,23.6,23.6,23.9,24.1,24.2,24.5,24.6,24.6,24.5,24.4,24.2
1949,24.2,23.9,24.0,24.0,24.0,24.0,23.8,23.9,24.0,23.9,23.9,23.8
1950,23.7,23.6,23.7,23.7,23.8,24.0,24.2,24.4,24.6,24.7,24.8,25.1
1951,25.5,25.9,26.0,26.0,26.1,26.1,26.1,26.1,26.3,26.4,26.5,26.6
1952,26.6,26.5,26.5,26.6,26.6,26.7,26.9,26.9,26.9,26.9,26.9,26.9


In [54]:
# Count number of months in dataset.
states = cpi.shape[1]
states

75

In [44]:
cpi

<tf.Tensor: shape=(75, 12), dtype=float32, numpy=
array([[ 23.8  ,  23.6  ,  23.6  ,  23.9  ,  24.1  ,  24.2  ,  24.5  ,
         24.6  ,  24.6  ,  24.5  ,  24.4  ,  24.2  ],
       [ 24.2  ,  23.9  ,  24.   ,  24.   ,  24.   ,  24.   ,  23.8  ,
         23.9  ,  24.   ,  23.9  ,  23.9  ,  23.8  ],
       [ 23.7  ,  23.6  ,  23.7  ,  23.7  ,  23.8  ,  24.   ,  24.2  ,
         24.4  ,  24.6  ,  24.7  ,  24.8  ,  25.1  ],
       [ 25.5  ,  25.9  ,  26.   ,  26.   ,  26.1  ,  26.1  ,  26.1  ,
         26.1  ,  26.3  ,  26.4  ,  26.5  ,  26.6  ],
       [ 26.6  ,  26.5  ,  26.5  ,  26.6  ,  26.6  ,  26.7  ,  26.9  ,
         26.9  ,  26.9  ,  26.9  ,  26.9  ,  26.9  ],
       [ 26.8  ,  26.7  ,  26.7  ,  26.8  ,  26.8  ,  26.9  ,  27.   ,
         27.1  ,  27.1  ,  27.2  ,  27.1  ,  27.   ],
       [ 27.1  ,  27.1  ,  27.   ,  27.   ,  27.1  ,  27.1  ,  27.1  ,
         27.1  ,  27.   ,  26.9  ,  27.   ,  26.9  ],
       [ 26.9  ,  26.9  ,  26.9  ,  26.9  ,  26.9  ,  26.9  ,  27.   ,
    

In [43]:
# Convert our dataframe to a tensorflow constant
cpi = tf.constant(np.array(cpi), tf.float32)

In [45]:
# Set number of hidden nodes.
hiddenNodes = 32

# Set number of nodes in latent state.
latentNodes = 5

# Define input layer for encoder.
encoderInput = tf.keras.layers.Input(shape = (states))

# Define hidden layer for the encoder.
encoderHidden = tf.keras.layers.Dense(hiddenNodes, 
activation = 'sigmoid')(encoderInput)

# Define the bottleneck layer.
latent = tf.keras.layers.Input(shape = (latentNodes))

In [46]:
# Define output layer for encoder.
encoded = tf.keras.layers.Dense(latentNodes, 
activation = 'sigmoid')(encoderHidden)

# Define hidden layer for decoder.
decoderHidden = tf.keras.layers.Dense(hiddenNodes, 
activation = 'sigmoid')(latent)

# Define output layer for decoder.
decoded = tf.keras.layers.Dense(states, 
activation = 'linear')(decoderHidden)


In [47]:
# Define separate models for encoder and decoder.
encoder = tf.keras.Model(encoderInput, encoded)
decoder = tf.keras.Model(latent, decoded)

# Define functional model for autoencoder.
autoencoder = tf.keras.Model(encoderInput, decoder(encoded))

# Compile the autoencoder.
autoencoder.compile(loss = 'mae', optimizer = 'adam')

# Train the autoencoder.
autoencoder.fit(cpi, cpi, epochs = 100)


Epoch 1/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 11/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 12/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 13/100
3/3 [==============================] - 0s 2ms/step - loss: nan
Epoch 14/100
3/3 [===

In [50]:
cpi[0:5]

<tf.Tensor: shape=(5, 12), dtype=float32, numpy=
array([[23.8, 23.6, 23.6, 23.9, 24.1, 24.2, 24.5, 24.6, 24.6, 24.5, 24.4,
        24.2],
       [24.2, 23.9, 24. , 24. , 24. , 24. , 23.8, 23.9, 24. , 23.9, 23.9,
        23.8],
       [23.7, 23.6, 23.7, 23.7, 23.8, 24. , 24.2, 24.4, 24.6, 24.7, 24.8,
        25.1],
       [25.5, 25.9, 26. , 26. , 26.1, 26.1, 26.1, 26.1, 26.3, 26.4, 26.5,
        26.6],
       [26.6, 26.5, 26.5, 26.6, 26.6, 26.7, 26.9, 26.9, 26.9, 26.9, 26.9,
        26.9]], dtype=float32)>

In [48]:
# Predict inflation series.
cpi_predicted = autoencoder.predict(cpi)

# Compute absolute deviations.
reconstructionLoss = pd.DataFrame(np.abs(cpi-cpi_predicted), 
columns = cNames)

# Generate latent state for inflation time series.
latentState = pd.DataFrame(encoder.predict(cpi))


TypeError: Index(...) must be called with a collection of some kind, 12 was passed